In [ ]:
def edit_distance_words(words1, words2):
    m = len(words1)
    n = len(words2)

    # Create a (m+1) x (n+1) matrix to store edit distances
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the first row and column
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    # Fill in the matrix using dynamic programming
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if words1[i - 1] == words2[j - 1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,  # Deletion
                dp[i][j - 1] + 1,  # Insertion
                dp[i - 1][j - 1] + cost,  # Substitution
            )

    # Backtrack to find the edit operations
    i, j = m, n
    operations = []
    while i > 0 and j > 0:
        if words1[i - 1] == words2[j - 1]:
            operations.append((words1[i - 1], "No change"))
            i -= 1
            j -= 1
        elif dp[i][j] == dp[i - 1][j] + 1:
            operations.append((words1[i - 1], "Delete"))
            i -= 1
        elif dp[i][j] == dp[i][j - 1] + 1:
            operations.append((words2[j - 1], "Insert"))
            j -= 1
        else:
            operations.append((words1[i - 1], "Substitute", words2[j - 1]))
            i -= 1
            j -= 1

    while i > 0:
        operations.append((words1[i - 1], "Delete"))
        i -= 1
    while j > 0:
        operations.append((words2[j - 1], "Insert"))
        j -= 1

    operations.reverse()
    return dp[m][n], operations

In [ ]:
import pandas as pd

manual_path = "Manual_x.xlsx"
automatic_path = "Automatic_x.xlsx"

manual_df = pd.read_excel(manual_path)
automatic_df = pd.read_excel(automatic_path)

manual_df['manual'] = manual_df['manual'].str.strip()
automatic_df['automatic'] = automatic_df['automatic'].str.strip()

manual_str = " ".join(manual_df['manual'].tolist())
automatic_str = " ".join(automatic_df['automatic'].tolist())

In [ ]:
distance, operations = edit_distance_words(automatic_str.split(), manual_str.split())

Write the result to a `.CSV` file:

In [ ]:
import csv

# Specify the CSV file path and column headers
csv_file = 'result.csv'
fieldnames = ['manual', 'automatic']

# Write data to the CSV file using DictWriter
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    cur = 0
    for manual_seg in manual_df["manual"]:
        length = len(manual_seg.strip().split())
        end_idx = cur
        while length:
            if operations[end_idx][1] != "Delete":
                length -= 1
            end_idx += 1

        automatic_seg_raw = operations[cur: end_idx]

        automatic_segment = ""
        for op in automatic_seg_raw:
            if op[1] != "Insert":
                automatic_segment += f" {op[0]}"
        cur = end_idx

        writer.writerow({"manual": manual_seg, "automatic": automatic_segment})

    remained_seg_raw = operations[end_idx:]
    remained_segment = ""
    if remained_seg_raw:
        for op in remained_seg_raw:
            if op[1] != "Insert":
                remained_segment += f" {op[0]}"

        writer.writerow({"manual": "", "automatic": remained_segment})
